In [4]:
import json
import string

def remove_punctuation(s):
    return s.translate(str.maketrans('', '', string.punctuation))
with open("official.json", "r") as fp:
    official = json.load(fp)

count = 0
headings = [
  "id", # Just a number
  "spotify_track_id", # track id 
  "title", # Display name
  "artist_name", # Display artist name
  "thumbnail", # Picture
  "aus", # 1 or 0
  "full_title", # title with artist
  "artist_name_string", # display artist name
#   "artists", # list of artist objects # id, title(artist name),  spotify_artist_id, 
]

csv_rows = [headings + ["artist_id_0", "artist_0", 
                        "artist_id_1", "artist_1", 
                        "artist_id_2", "artist_2", 
                        "artist_id_3", "artist_3", 
                        "artist_id_4", "artist_4"
                        ]]

"""
  {
    "id": "0YuYaObSGXPQfNmzNKDGEK",
    "name": "Won 't Stop [Ft. Jess Mauboy]",
    "artist": "3%",
    "game_rule": "",
    "full_title": "Won' t Stop [Ft. Jess Mauboy] - 3%",
    "artist_ids": "4CnzGjdsHYGHgmsDk1tNwg|6rHWAH6F4mr2AViSxMV673|",
    "searchable_name": "wontstopftjessmauboy",
    "searchable_artist": "3%"
  }
"""
json_output = []


for key, val in official.items():
    csv_row = []
    data = {
        "id": val["spotify_track_id"] if val["spotify_track_id"] else remove_punctuation("".join(val["title"].lower().split())),
        "name": val["title"],
        "artist": val["artist_name"],
        "game_rule": "",
        "full_title": val["full_title"],
        "artists": [
            {"id": artist["spotify_artist_id"], 
             "name": artist["title"],
               "searchable_name": remove_punctuation("".join(artist["title"].lower().split()))
            }  for artist in val["artists"]],
        "searchable_name": remove_punctuation("".join(val["title"].lower().split())),
        "searchable_artist": remove_punctuation("".join(val["artist_name"].lower().split())),
    }
    json_output.append(data)
    for heading in headings:
        if heading not in val:
            print(f"Missing heading {heading} for {val['title']} by {val['artist_name']}")
            csv_row.append("")
        else:
            csv_row.append(val[heading])
    if len(val["artists"]) > 5:
        print(f"Too many artists for {val['title']} by {val['artist_name']}")
    for artist in val["artists"]:
        csv_row.append(artist["spotify_artist_id"])
        csv_row.append(artist["title"])
    csv_rows.append(csv_row)

with open("frontend/src/assets/official_output.json", "w") as fp:
    json.dump(json_output, fp, indent=2)

import csv
with open("official.csv", "w", newline="", encoding="utf-16") as fp:
    writer = csv.writer(fp)
    writer.writerows(csv_rows)